In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm

Using TensorFlow backend.


In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [4]:
sales = pd.read_csv("sales_train.csv")

In [5]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [6]:
pre_df = sales.copy()
pre_df = pre_df.pivot_table(
    index=['shop_id', 'item_id'],
    values=['item_cnt_day'],
    columns=['date_block_num'],
    fill_value=0,
    aggfunc='sum'
).reset_index()

pre_df

shop_id item_id item_cnt_day                        ...        \
date_block_num                            0   1  2  3  4  5  6  7  ... 24 25   
0                    0      30            0  31  0  0  0  0  0  0  ...  0  0   
1                    0      31            0  11  0  0  0  0  0  0  ...  0  0   
2                    0      32            6  10  0  0  0  0  0  0  ...  0  0   
3                    0      33            3   3  0  0  0  0  0  0  ...  0  0   
4                    0      35            1  14  0  0  0  0  0  0  ...  0  0   
...                ...     ...          ...  .. .. .. .. .. .. ..  ... .. ..   
424119              59   22154            1   0  0  0  0  0  0  0  ...  0  0   
424120              59   22155            0   0  0  0  0  0  1  0  ...  0  0   
424121              59   22162            0   0  0  0  0  0  0  0  ...  0  9   
424122              59   22164            0   0  0  0  0  0  0  0  ...  0  2   
424123              59   22167            0   0  0  0  0  0  0  0  ...  0  0   

                                        
date_block_num 26 27 28 29 30 31 32 33  
0               0  0  0  0  0  0  0  0  
1               0  0  0  0  0  0  0  0  
2               0  0  0  0  0  0  0  0  
3               0  0  0  0  0  0  0  0  
4               0  0  0  0  0  0  0  0  
...            .. .. .. .. .. .. .. ..  
424119          0  0  0  0  0  0  0  0  
424120          0  0  0  0  0  0  0  0  
424121          4  1  1  0  0  1  0  0  
424122          1  2  0  0  1  0  0  0  
424123          0  0  0  0  0  0  0  0  

[424124 rows x 36 columns]

In [7]:
test = pd.read_csv('test.csv')

In [8]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [9]:
full_train_df = test.copy()
full_train_df = full_train_df.merge(pre_df, how='left', on=['shop_id', 'item_id']).fillna(0).drop(
    ['ID', 'shop_id', 'item_id'], axis=1)
full_train_df

/home/nitin/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/home/nitin/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X_train, y_train = full_train_df.values[:,:-2], full_train_df.values[:, -2:-1].ravel()
X_valid, y_valid = full_train_df.values[:,1:-1], full_train_df.values[:, -1:].ravel()
X_test = full_train_df.values[:, 2:]

In [11]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=100)

In [12]:
subsequences = 2
timesteps = X_train.shape[1]//subsequences

In [13]:
X_train = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_valid = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_test = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))

In [14]:
modelConvLSTM = Sequential()
modelConvLSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='relu', return_sequences=True,input_shape=(X_train.shape[1], 1, X_train.shape[3], X_train.shape[4])))
modelConvLSTM.add(ConvLSTM2D(filters=16, kernel_size=(1,2), activation='relu'))
modelConvLSTM.add(Flatten())
modelConvLSTM.add(Dense(1))
modelConvLSTM.compile(optimizer='adam', loss='mse')
history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=1000, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))

Train on 214200 samples, validate on 214200 samples
Epoch 1/1000
214200/214200 [==============================] - 32s 147us/step - loss: 3.0294 - val_loss: 24.6308
Epoch 2/1000
214200/214200 [==============================] - 29s 136us/step - loss: 2.7314 - val_loss: 21.6766
Epoch 3/1000
214200/214200 [==============================] - 29s 136us/step - loss: 2.0348 - val_loss: 22.3966
Epoch 4/1000
214200/214200 [==============================] - 29s 136us/step - loss: 2.2282 - val_loss: 23.9522
Epoch 5/1000
214200/214200 [==============================] - 29s 137us/step - loss: 1.8954 - val_loss: 24.5995
Epoch 6/1000
214200/214200 [==============================] - 29s 137us/step - loss: 2.2853 - val_loss: 27.9765
Epoch 7/1000
214200/214200 [==============================] - 30s 138us/step - loss: 2.8939 - val_loss: 24.3300
Epoch 8/1000
214200/214200 [==============================] - 30s 138us/step - loss: 2.2931 - val_loss: 22.1912
Epoch 9/1000
214200/214200 [========================

214200/214200 [==============================] - 30s 138us/step - loss: 1.9021 - val_loss: 23.5911
Epoch 74/1000
214200/214200 [==============================] - 28s 133us/step - loss: 1.8072 - val_loss: 22.0052
Epoch 75/1000
214200/214200 [==============================] - 28s 133us/step - loss: 1.7960 - val_loss: 21.7292
Epoch 76/1000
214200/214200 [==============================] - 29s 134us/step - loss: 1.8261 - val_loss: 22.3621
Epoch 77/1000
214200/214200 [==============================] - 30s 139us/step - loss: 1.8775 - val_loss: 21.7870
Epoch 78/1000
214200/214200 [==============================] - 30s 142us/step - loss: 1.9167 - val_loss: 22.1423
Epoch 79/1000
214200/214200 [==============================] - 30s 142us/step - loss: 1.9971 - val_loss: 22.6031
Epoch 80/1000
214200/214200 [==============================] - 32s 150us/step - loss: 1.9086 - val_loss: 22.1008
Epoch 81/1000
214200/214200 [==============================] - 32s 147us/step - loss: 1.8474 - val_loss: 22.59

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/nitin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-d4b6cc235070>", line 7, in <module>
    history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=1000, verbose=1, callbacks=[early_stop], validation_data = (X_valid, y_valid))
  File "/home/nitin/anaconda3/lib/python3.7/site-packages/keras/engine/training.py", line 1239, in fit
    validation_freq=validation_freq)
  File "/home/nitin/anaconda3/lib/python3.7/site-packages/keras/engine/training_arrays.py", line 196, in fit_loop
    outs = fit_function(ins_batch)
  File "/home/nitin/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/backend.py", line 3727, in __call__
    outputs = self._graph_fn(*converted_inputs)
  File "/home/nitin/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py", line 1551, in __call

KeyboardInterrupt: 

In [15]:
ConvLSTMpred = modelConvLSTM.predict(X_valid, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
ConvLSTMresults = metrics(ConvLSTMpred, y_valid)
ConvLSTMresults

{'Explained Variance Score': 0.2819672021063866,
 'Max Error': 2008.996597290039,
 'Mean Absolute Error': 0.35260383782009175,
 'Mean Squared Error': 22.127535199605585,
 'Root Mean Squared Error': 4.703991411514862,
 'Median Absolute Error': 0.08697494864463806,
 'R² Score': 0.2818809565761753,
 'Mean Absolute Percentage Error': 242.8300179548552}